In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score,confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
# Utiliy Functions
def min_max_data_normalize(data_train,data_test):
    data_min=data_train.min()
    data_max=data_train.max()
    normal_data_train=(data_train-data_min)/(data_max-data_min)
    normal_data_test=(data_test-data_min)/(data_max-data_min)
    return normal_data_train,normal_data_test

In [3]:
def data_processed():
    '''
    Used to read & normalize processed test and train data
    '''
    data_train=pd.read_csv("data/2022-02-06_LOANS_TRAIN.csv")
    data_test=pd.read_csv("data/2022-02-06_LOANS_TEST.csv")
    y_train=data_train['loan_status']
    data_train.drop('loan_status',1,inplace=True)
    normal_data_train,normal_data_test=min_max_data_normalize(data_train,data_test)
    return normal_data_train,normal_data_test,y_train

def AUC_score(y_ground_truth,y_predicted_probability):
    return roc_auc_score(y_ground_truth, y_predicted_probability)

def to_submission(y_test_predicted_probability):
    y_test=pd.DataFrame(y_test_predicted_probability,columns=['loan_status'])
    y_test.to_csv('data/submission.csv', index_label='id')
    return

In [4]:
X_train,X_test,Y_train=data_processed()
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.80, random_state=42)

In [10]:
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, Y_train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [7]:
clf = SVC(gamma='auto',probability=True)
clf.fit(X_train, Y_train)

SVC(gamma='auto', probability=True)

In [11]:
Y_val_pred_prob=clf.predict_proba(X_val)[:,1]
Y_val_pred=clf.predict(X_val)
AUC_score(Y_val,Y_val_pred)

0.5014402052502848

In [12]:
confusion_matrix(Y_val,Y_val_pred)

array([[133533,    130],
       [ 24044,     93]])

In [13]:
Y_test_pred_prob=clf.predict_proba(X_test)[:,1]

In [14]:
to_submission(Y_test_pred_prob)